In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Loding the dataset into pandas dataframe.
#df = pd.read_csv('../Data/data_banknote_authentication.csv')

# the code is changed to read data from a github url
url='https://raw.githubusercontent.com/sujitcl/code/main/Data/data_banknote_authentication.csv'

df=pd.read_csv(url)


In [ ]:
# print all the available features.
df.columns

Index(['variance', 'skew', 'kurtosis', 'entropy', 'authentic'], dtype='object')

In [ ]:
# Check for nulls.
df.columns[df.isnull().any()]

Index([], dtype='object')

In [ ]:
# Count the number of malignants and benigns in the dataset.
df['authentic'].value_counts()

0    762
1    610
Name: authentic, dtype: int64

In [ ]:
df.sample(n=5, random_state=55).sort_values(['variance'])

,variance,skew,kurtosis,entropy,authentic
1180,-2.21830,-1.2540,2.9986,0.36378,1
769,-0.89409,3.1991,-1.8219,-2.94520,1
1353,0.11592,3.2219,-3.4302,-2.84570,1
239,2.39520,9.5083,-3.1783,-3.00860,0
722,4.84510,8.1116,-2.9512,-1.47240,0


## Create the Dataframe of features (X) and the target (Y) variables

In [ ]:
# Load the features to a variable X
# X is created by simply dropping the diagnosis column and retaining all others
X = df.drop('authentic', axis = 1)

# Load the target variable to y
y = df['authentic']

## Split Test Train

**> Train-Test split -** We split our data into two parts, namely, the train set and the test set (ideally its a 70-30 train-test split which is upto you). We then try to build our function f(x) (aka model) using the train set and see how well it does on the test set.   

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

## Create an Instance of the classifier and train it.

In [ ]:
# Let's create an instance for the LogisticRegression model and then train it with the training set.
from sklearn.ensemble import RandomForestClassifier
Classifier = RandomForestClassifier(random_state=0, min_samples_leaf=10)
Classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## Get the Predictions

In [ ]:
# Getting predictions from the model 
y_test_hat = Classifier.predict(X_test)

# Compare the predicted values with the actuals.
Results = pd.DataFrame({'Actual': y_test})
column = pd.DataFrame({'Predictions': y_test_hat})
Results = Results.join(column.set_index(Results.index))
Results.head(5)

,Actual,Predictions
1240,1,1
703,0,0
821,1,1
1081,1,1
37,0,0


### 1. The accuracy scores

It is simply calculated as number of classes predicted right divided by total number of samples.

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_test_hat))

0.9878640776699029


In [ ]:
# Get the predictions from the model for the training set. 
y_train_hat = Classifier.predict(X_train)
print(accuracy_score(y_train, y_train_hat))

0.9885416666666667


### 2. The confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score
 
cm = confusion_matrix(y_test, y_test_hat)
print(cm)

[[231   4]
 [  1 176]]


In [ ]:
# Assigning Variables for convinience
TN = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TP = cm[1][1]

recall = TP / float(FN + TP)
print("recall:", recall)

precision = TP / float(TP + FP)
print("precision:", precision)

specificity = TN / (TN + FP)
print("specificity:", specificity)

recall: 0.9943502824858758
precision: 0.9777777777777777
specificity: 0.9829787234042553


### Feature importance

In [ ]:
feature_importances = pd.DataFrame(Classifier.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',                                                                 ascending=False)

feature_importances

,importance
variance,0.599044
skew,0.223925
kurtosis,0.133505
entropy,0.043526
